In [3]:
import rasterio as rio
import numpy as np
import pandas as pd
import os
from glob import glob
from datetime import datetime

input_base_path = r"E:\LULC\raw_data\1_landsat\6_landsat8_mosaic"
output_base_path = r"E:\LULC\features\01_landsat8\train"
target_file = r"E:\LULC\raw_data\0_groundtruth\SACLAJ_2014_2016_catclearned.csv"

seasons = ["winter", "spring", "summer", "autumn"]
target_df = pd.read_csv(target_file)
target_list = []

for i in range(len(target_df)):
    sacla_id = "{:05}".format(target_df["id"][i])
    lon = float(target_df["target_lon"][i])
    lat = float(target_df["target_lat"][i])
    target = target_df["category"][i]
    target_list.append([sacla_id, lon, lat, target])

N_list = [11, 21, 31]

start = datetime.now()

for N in N_list:
    print(f"{N}*{N} is starting...")
    for season in seasons:
        print(f"{season} is starting...")
        input_path = os.path.join(input_base_path, season)
        bands = glob(input_path + "/*.tif")

    #   各シーズンごとの各バンドのパス: band
        for band in bands:
            with rio.open(band) as dataset:

                try:
                    os.makedirs(os.path.join(output_base_path, f"{N}x{N}"))
                except FileExistsError:
                    pass 
                output_file = os.path.join(output_base_path, f"{N}x{N}")
                band_name = os.path.basename(band).split(".")[0]

                for i, (sacla_id, lon, lat, target) in enumerate(target_list): 

                    py, px = dataset.index(lon, lat)
                    window = rio.windows.Window(px - N//2, py - N//2, N, N)
                    clip = dataset.read(window=window)

                    meta = dataset.meta
                    meta['width'], meta['height'] = N, N
                    meta['transform'] = rio.windows.transform(window, dataset.transform)
                    
                    
                    lon = "{:.6f}".format(round(lon, 6))
                    lat = "{:.6f}".format(round(lat, 6))
                    output_path = os.path.join(output_file, f"{sacla_id}_{lon}_{lat}_{season}_{band_name}_{target}.tif")
                    with rio.open(output_path, 'w', **meta) as dst:
                        dst.write(clip)
    
    end = datetime.now()
    print(f"{N}*{N} is finished... time is {end-start} seconds.")

11*11 is starting...
winter is staeting...
spring is staeting...
summer is staeting...
autumn is staeting...
11*11 is finished... time is -1 day, 12:04:20.744875 seconds.
21*21 is starting...
winter is staeting...
spring is staeting...
summer is staeting...
autumn is staeting...
21*21 is finished... time is -1 day, 5:40:24.728686 seconds.
31*31 is starting...
winter is staeting...
spring is staeting...
summer is staeting...
autumn is staeting...
31*31 is finished... time is -2 days, 23:16:35.209190 seconds.
